First import the prerequisites, pandas, sqlalchemy and plotly. Install them with "pip install" or Anaconda if that isn't done already.

In [3]:
import pandas as pd 
from sqlalchemy import create_engine

import plotly.plotly as py
from plotly.graph_objs import *

Import the SQLite database so that it can be read with Pandas.

Note the three slashes instead of just one for a relative path.

In [ ]:
disk_engine = create_engine("sqlite:///data/turbine_locations.db")

In [ ]:
df = pd.read_sql_query("Select lat_DD, long_DD, count(*) from data group by lat_DD, long_DD ", disk_engine)

Let's query the dataframe to see everything works fine so far.

In [4]:
df = pd.read_csv("turbine_locations.csv") 
print(df)

        xFID  unique_id                               site_name  total_turb  \
0          0        982                  unknown Gilliam County           2   
1          1       1065                  unknown Gilliam County           2   
2          2       1895                     Banner Wind Project           2   
3          3       1897                     Banner Wind Project           2   
4          4       2608  unknown Tehachapi Wind Resource Area 1         549   
5          5       2628  unknown Tehachapi Wind Resource Area 1         549   
6          6       2661  unknown Tehachapi Wind Resource Area 1         549   
7          7       2797                               Sky River           5   
8          8       2916                               Sky River           5   
9          9       2917                               Sky River           5   
10        10       2997                               Sky River           5   
11        11       3127  unknown Tehachapi Wind Reso

In [6]:
print(df["lat_DD"].values)

[ 45.653994  45.663494  64.569901 ...,  41.687993  41.689393  41.690793]


Now let's do a basic graph with Plotly. First setup your account for Plotly graphing here if you haven't used Plotly before : 

https://plot.ly/python/getting-started/

Do not skip this step as Plotly requires API authentication prior to graphing.

In [7]:
mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjaXhzYTB0bHcwOHNoMnFtOWZ3YWdreDB3In0.pjROwb9_CEuyKPE-x0lRUw'

trace = Scattermapbox(
        lat=df['lat_DD'].values,
        lon=df['long_DD'].values,
        mode='markers',
        marker=Marker(
            size=5
        ),
    )

data = Data([trace])

layout = Layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 1", validate=False)

/Users/admin/anaconda/lib/python3.5/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [ ]:
mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjaXhzYTB0bHcwOHNoMnFtOWZ3YWdreDB3In0.pjROwb9_CEuyKPE-x0lRUw'

trace = Scattermapbox(
        lat=df['lat_DD'].values,
        lon=df['long_DD'].values,
        mode='markers',
        marker=Marker(
            color="#FFFFFF",
            size=5
        ),
    )

data = Data([trace])

layout = Layout(
    annotations=Annotations([
        Annotation(
            x=0.00,
            y=1.07,
            showarrow=False,
            text="Map 1 : Visualization of wind turbine locations in the US",
            xref='paper',
            yref='paper'
        )
    ]),
    font=Font(family="Droid Sans", size=14, color="FFFFFF"),
    autosize=True,
    hovermode='closest',
    paper_bgcolor='rgb(40, 40, 40)',
    margin=Margin(
        t=60,
        l=20,
        b=20,
        r=20,
        pad=0
    ),
    title = "",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5,
        style="dark"
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 1 styled", validate=False)

Looks much better already. Now let's add color and vary the size of the points with respect to blade length.

In [ ]:
#df = pd.read_sql_query("Select lat_DD, long_DD, blade_l, count(*) from data group by lat_DD, long_DD, blade_l", disk_engine)

mapbox_access_token = 'pk.eyJ1IjoiamFja2x1byIsImEiOiJjaXhzYTB0bHcwOHNoMnFtOWZ3YWdreDB3In0.pjROwb9_CEuyKPE-x0lRUw'

trace = Scattermapbox(
    lat=df['lat_DD'].values,
    lon=df['long_DD'].values,
    mode='markers',
    marker=Marker(
        color=[max(float(i), 5) for i in df['blade_l'].values],
        colorscale=[[0, 'rgb(221,42,145)'], [0.35, 'rgb(177,77,236)'], [0.5, 'rgb(118,117,237)'], [0.6, 'rgb(46,142,191)'], [0.7, 'rgb(11,152,121)'], [1, 'rgb(19,152,99)']],
        reversescale=True,
        showscale=True,
        autocolorscale=False,
        colorbar=ColorBar(
            title='Turbine<br>blade<br>length (m)'
        ),
        size=[max(float(i)/5, 5) for i in df['blade_l'].values],
        opacity=0.8
      
    ),
)
 
data = Data([trace])

layout = Layout(
    annotations=Annotations([
        Annotation(
            x=0.00,
            y=1.07,
            showarrow=False,
            text="Map 2 : Visualization of wind turbine locations and their size in the US",
            xref='paper',
            yref='paper'
        )
    ]),
    font=Font(family="Droid Sans", size=14, color="FFFFFF"),
    autosize=True,
    hovermode='closest',
    paper_bgcolor='rgb(40, 40, 40)',
    margin=Margin(
        t=60,
        l=20,
        b=20,
        r=20,
        pad=0
    ),
    title = "",
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=42,
            lon=-95
        ),
        pitch=0,
        zoom=5,
        style="dark"
    ),
)

fig = Figure(data=data,layout=layout)
plot_url = py.plot(fig, filename = "Mapbox 2", validate=False)
